In [3]:
import pathlib
import tensorflow as tf

# Download the dataset provided by Anki, https://www.manythings.org/anki/
Using the keras library, we download the text file representing the dataset

In [4]:
text_file = tf.keras.utils.get_file(
    fname="fra-eng.zip",
    origin="http://storage.googleapis.com/download.tensorflow.org/data/fra-eng.zip",
    extract=True,
)

3423204/3423204 [==============================] - 1s 0us/step


In [5]:
# show where the file is located now
text_file = pathlib.Path(text_file).parent / "fra.txt"
print(text_file)

/Users/adeyemialadesawe/.keras/datasets/fra.txt
